In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/total_segmentator/AutoPET/"

/content/drive/MyDrive/total_segmentator/AutoPET


In [ ]:
!pip install monai
!pip install torchmetrics
!pip install kornia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.7/705.7 kB 8.9 MB/s eta 0:00:00


In [ ]:
# Image Shape from dataloader: N3HW
# Mask Shape from dataloader: NHW
# Model Prediction Shape: N2HW

import numpy as np
from argparse import Namespace
import matplotlib.pyplot as plt

import torch
import monai
import torchmetrics as tm
import torchsummary
from torch.utils.data import DataLoader
from torchvision.transforms import Compose
from torchvision.transforms.functional import InterpolationMode
from monai.losses import DiceLoss
import PIL.Image
import os
from unet import GatedUNet

from dataset2 import InPaintingDataset2
from dataset2 import RandContrast, RandVerFlip, RandHorFlip, RandRotation, Resize, ToTensor, Normalize, Crop
from loss import valid_loss_fn, hole_loss_fn, tv_loss_fn, PerceptualLoss, StyleLoss, LPLoss

In [ ]:
batch_size = 16
image_size = (512, 512)

train_transform = Compose([
                           ToTensor(),
                           Crop()])

valid_transform = Compose([ToTensor(),
                           Crop()])

train_dir = "/content/drive/MyDrive/example_data_autopet/train"

train_set = InPaintingDataset2(train_dir, transform=train_transform)

train_loader = DataLoader(train_set, batch_size, shuffle=False)

counter = 0
patience = 7
save_model = True
model_id = abs(round(np.random.normal(0, 1), 5))

lr = 0.0001
num_epochs = 600
best_valid_loss = float("inf")

model_name = "GatedUnet"
dataset_name = "AutoPET"
pretrain_model = True
loss_log = "combine Loss"
print(model_id)


1.24797


**For testting the dataloader**

In [ ]:
for i, data in enumerate(train_loader):
  img, seg, mask = data
  print("image shape is", img.shape)
  print("seg shape is", seg.shape)
  print("mask shape is", mask.shape)


image shape is torch.Size([12, 2, 384, 384])
seg shape is torch.Size([12, 2, 384, 384])
mask shape is torch.Size([12, 2, 384, 384])
image shape is torch.Size([12, 2, 384, 384])
seg shape is torch.Size([12, 2, 384, 384])
mask shape is torch.Size([12, 2, 384, 384])


KeyboardInterrupt: ignored

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = GatedUNet(in_channels=2, out_channels=2, bilinear=False)
model.to(device)
model.train()
#defining loss function
#loss_fn = DiceLoss(include_background=False, to_onehot_y=False, sigmoid=True)
mse_loss = torch.nn.MSELoss()
l1_loss = torch.nn.L1Loss()
perceptual_loss_fn = PerceptualLoss(device=device)
style_loss_fn = StyleLoss(device=device)
lp_loss_fn = LPLoss(max_level=3)
optim = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.001)
acc_fn = tm.classification.BinaryAccuracy().to(device)
pre_fn = tm.classification.BinaryPrecision().to(device)
rec_fn = tm.classification.BinaryRecall().to(device)

cuda


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
for epoch in range(num_epochs):

    model.train()
    train_losses, valid_losses = [], []
    for i, data in enumerate(train_loader):

        imgs, labels, masks = data
        imgs = imgs.to(device).to(torch.float32)# images shape Nx2x384x384
        labels = labels.to(device).to(torch.float32)#labels shape Nx2x384x384
        masks = masks.to(device)#masks shape Nx2x384x384

        optim.zero_grad()
        preds = model(imgs)  # predictions in the shape of Nx1x512x512
        valid_loss = valid_loss_fn(preds, labels, masks)
        hole_loss = hole_loss_fn(preds, labels, masks)
        lp_loss = lp_loss_fn(preds, labels)
        loss = 30 * valid_loss + 240 * hole_loss + 20 * lp_loss
        loss.backward()
        optim.step()
        train_losses.append(loss.detach().item())

        model.eval()
        with torch.no_grad():
          val_preds = model(imgs)
          val_valid_loss = valid_loss_fn(val_preds.detach(), labels.detach(), masks.detach())
          val_hole_loss = hole_loss_fn(val_preds.detach(), labels.detach(), masks.detach())
          val_lp_loss = lp_loss_fn(val_preds.detach(), labels.detach())
          val_loss = 30 * val_valid_loss + 240 * val_hole_loss + 20 * val_lp_loss
          valid_losses.append(val_loss.detach().item())

    ave_train_loss = sum(train_losses) / len(train_losses)
    ave_valid_loss = sum(valid_losses) / len(valid_losses)

    if save_model and epoch > 30 and epoch % 2 == 0:
      torch.save(model.state_dict(), f"/content/drive/MyDrive/DRIU/models/DRIU/model_{model_id}_{epoch}.pt")

    print(f"Epoch {epoch} train loss: ", ave_train_loss)
    print(f"Epoch {epoch} valid loss: ", ave_valid_loss)
    print()

    acc_fn.reset()
    pre_fn.reset()
    rec_fn.reset()


Epoch 0 train loss:  162.41899217878068
Epoch 0 valid loss:  162.05252402169364

Epoch 1 train loss:  158.36817169189453
Epoch 1 valid loss:  160.12140001569475

Epoch 2 train loss:  152.79448754446847
Epoch 2 valid loss:  155.04108210972376

Epoch 3 train loss:  79.35283497401646
Epoch 3 valid loss:  72.32452733176095

Epoch 4 train loss:  22.78074053355626
Epoch 4 valid loss:  14.994573729378837

Epoch 5 train loss:  21.883156503949845
Epoch 5 valid loss:  14.991040638514928

Epoch 6 train loss:  21.099161284310476
Epoch 6 valid loss:  15.335362706865583

Epoch 7 train loss:  19.84528875350952
Epoch 7 valid loss:  14.80609825679234

Epoch 8 train loss:  19.231173787798202
Epoch 8 valid loss:  14.73001595905849

Epoch 9 train loss:  18.76236363819667
Epoch 9 valid loss:  14.733784130641393

Epoch 10 train loss:  18.422977175031388
Epoch 10 valid loss:  14.71890504019601

Epoch 11 train loss:  18.116809163774764
Epoch 11 valid loss:  14.710119247436523

Epoch 12 train loss:  17.8988253

KeyboardInterrupt: ignored

**Test**

In [ ]:
batch_size = 4

test_dir = "/content/drive/MyDrive/example_data_autopet/test"

test_set = InPaintingDataset2(test_dir, transform=train_transform)

test_loader = DataLoader(test_set, batch_size, shuffle=False)

model_path = "/content/drive/MyDrive/DRIU/models/DRIU/model_2.59424_350.pt"
model = GatedUNet(in_channels=2, out_channels=2, bilinear=False)
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()



GatedUNet(
  (down1): Down(
    (gated_conv): GatedConvolution(
      (conv): Conv2d(2, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (gate_conv): Conv2d(2, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (sigmoid): Sigmoid()
    )
    (BN_layer): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (down2): Down(
    (gated_conv): GatedConvolution(
      (conv): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
      (gate_conv): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
      (sigmoid): Sigmoid()
    )
    (BN_layer): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (down3): Down(
    (gated_conv): GatedConvolution(
      (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (gate_conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (sigmoid): Sigmoid()

In [ ]:
imgs_save_path = "/content/drive/MyDrive/example_data_autopet/qualitative_results/imgs"
gts_save_path = "/content/drive/MyDrive/example_data_autopet/qualitative_results/gts"
preds_save_path = "/content/drive/MyDrive/example_data_autopet/qualitative_results/preds"

False


In [ ]:
test_losses = []
for i, data in enumerate(test_loader):

  test_imgs, test_labels, test_masks = data
  test_imgs = test_imgs.to(device).to(torch.float32)# images shape Nx1x512x512
  test_labels = test_labels.to(device).to(torch.float32)#labels shape Nx1x512x512
  test_masks = test_masks.to(device)#masks shape Nx1x512x512
  test_preds = model(test_imgs)
  print(test_preds[0][0].max())
  print(test_labels[0][0].max())
  print(test_imgs.shape)
  test_valid_loss = valid_loss_fn(test_preds.detach(), test_labels.detach(), test_masks.detach())
  test_hole_loss = hole_loss_fn(test_preds.detach(), test_labels.detach(), test_masks.detach())
  test_loss = 30 * test_valid_loss + 240 * test_hole_loss
  test_losses.append(test_loss.detach().item())
  #values, counts = torch.unique(test_labels, return_counts=True)
  #print(values, counts)
  for j in range(test_imgs.shape[0]):
    saved_imgs = (test_imgs[j][1] * 255).squeeze().cpu().numpy().astype(np.uint8)
    saved_preds = (test_preds[j][1] * 255).squeeze().detach().cpu().numpy().astype(np.uint8)
    saved_gts = (test_labels[j][1] * 255).squeeze().cpu().numpy().astype(np.uint8)

    saved_imgs_image = PIL.Image.fromarray(saved_imgs)
    saved_preds_image = PIL.Image.fromarray(saved_preds)
    saved_gts_image = PIL.Image.fromarray(saved_gts)

    save_path_imgs = os.path.join(imgs_save_path, str(i*batch_size+j) + ".png")
    save_path_preds = os.path.join(preds_save_path, str(i*batch_size+j) + ".png")
    save_path_gts = os.path.join(gts_save_path, str(i*batch_size+j) + ".png")


    saved_imgs_image.save(save_path_imgs)
    saved_preds_image.save(save_path_preds)
    saved_gts_image.save(save_path_gts)

ave_test_loss = sum(test_losses) / len(test_losses)
print("average test loss is", ave_test_loss)

tensor(0.9512, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(1., device='cuda:0')
torch.Size([1, 2, 384, 384])
average test loss is 0.6204289197921753
